<a href="https://colab.research.google.com/github/FarrellMinette/Hackathon/blob/main/BigQuery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Setup
from google.colab import auth
from google.cloud import bigquery
from google.colab import data_table

project_id = "hackathon-1123-395609"
data = "hackathon_data"
tables = "hackathon_tables"
table = "drivers"

project = project_id
# location = '|location|' # Location inserted based on the query results selected to explore
client = bigquery.Client(project=project)
data_table.enable_dataframe_formatter()
auth.authenticate_user()

In [ ]:
# Run sql query
def run_sql(sql):
    script = sql
    query_job = client.query(sql)
    result = query_job.result()
    return result

def test():
    sql = f"""Select * from {project_id}.{tables}.driver_backup;"""
    results = run_sql(sql)
    for result in results:
        print(result)

test()

Row((203444726, 0.0, 0, 17, 0.0, 0, 0, 0.0, 0.0, 0, 311, 0, 78, 74, 0), {'VehicleID': 0, 'TotalClaimsCost': 1, 'NumberOfClaims': 2, 'OverSpeed': 3, 'IdleRatio': 4, 'GpsLostCount': 5, 'TelematicsOffCount': 6, 'AverageNmrStopsPerDay': 7, 'AverageDistPerStop': 8, 'EmergencyBraking': 9, 'NormalBraking': 10, 'HarshBraking': 11, 'dangerousTimes': 12, 'DangerousAreas': 13, 'CriticalBraking': 14})
Row((139599033, 0.0, 0, 0, 0.0, 0, 15, 0.0, 0.0, 0, 30, 0, 0, 0, 0), {'VehicleID': 0, 'TotalClaimsCost': 1, 'NumberOfClaims': 2, 'OverSpeed': 3, 'IdleRatio': 4, 'GpsLostCount': 5, 'TelematicsOffCount': 6, 'AverageNmrStopsPerDay': 7, 'AverageDistPerStop': 8, 'EmergencyBraking': 9, 'NormalBraking': 10, 'HarshBraking': 11, 'dangerousTimes': 12, 'DangerousAreas': 13, 'CriticalBraking': 14})
Row((140138339, 0.0, 0, 0, 0.0, 0, 0, 10.9659090909091, 21672.1689119171, 0, 0, 0, 1310, 0, 0), {'VehicleID': 0, 'TotalClaimsCost': 1, 'NumberOfClaims': 2, 'OverSpeed': 3, 'IdleRatio': 4, 'GpsLostCount': 5, 'Telematic

In [ ]:
table = "drivers"
## Create the 'driver_backup' table
def create_driver():
    sql_create_driver_table = f"""
    CREATE OR REPLACE TABLE `{project_id}.{tables}.{table}` (
        vehicleid INT64
    )
    """
    query_job = client.query(sql_create_driver_table)
    query_job.result()


## Select all the vehicle ids and store in table_names
def select_table_names():
    result = run_sql(f"""SELECT table_id FROM `{project_id}.{data}.__TABLES__` """)
    table_names = [row.table_id for row in result]
    return table_names

## Add vehicle ids to the driver_backup table in hackathon_tables
# Loop through table names and extract vehicleid using SQL
def insert_vehicleid():
    table_names = select_table_names()
    for table_name in table_names:
        results = run_sql(f"""
        SELECT distinct(vehicleid) FROM `{project_id}.{data}.{table_name}`
        """)

        # Insert vehicleid values into the driver_backup table
        for row in results:
            vehicleid = row.vehicleid
            run_sql(f"""INSERT INTO {project_id}.{tables}.{table} (vehicleid) VALUES ({vehicleid})""")

def insert_over_speed_limit():
    table_names = select_table_names()
    run_sql(f"""Alter table {project_id}.{tables}.{table} ADD COLUMN IF NOT EXISTS overSpeed INT64""")
    for table_name in table_names:
        results = run_sql(f"""UPDATE `{project_id}.{tables}.{table}`
                            SET overSpeed = (
                            SELECT COUNT(*)
                            FROM `{project_id}.{data}.{table_name}`
                            WHERE speed > 1.1*road_speed
                            )
                            WHERE vehicleid = {table_name};
                        """)

def insert_gps_lost_count():
    table_names = select_table_names()
    sql = f"""Alter table {project_id}.{tables}.{table} ADD COLUMN IF NOT EXISTS gps_lost_count INT64;"""
    run_sql(sql)
    for table_name in table_names:
        sql = f"""UPDATE `{project_id}.{tables}.{table}`
                            SET gps_lost_count = (
                            SELECT COUNT(*)
                            FROM `{project_id}.{data}.{table_name}`
                            WHERE event_description='GPS_Lost'
                            )
                            WHERE vehicleid = {table_name};
                        """
        results = run_sql(sql)

def insert_telematics_off_count():
    table_names = select_table_names()
    sql = f"""Alter table {project_id}.{tables}.{table} ADD COLUMN IF NOT EXISTS telematics_off_count INT64;"""
    run_sql(sql)
    for table_name in table_names:
        sql = f"""UPDATE `{project_id}.{tables}.{table}`
                            SET telematics_off_count = (
                            SELECT COUNT(*)
                            FROM `{project_id}.{data}.{table_name}`
                            WHERE event_description='Power OFF'
                            )
                            WHERE vehicleid = {table_name};
                        """
        results = run_sql(sql)

def insert_normal_braking():
    table_names = select_table_names()
    sql = f"""Alter table {project_id}.{tables}.{table} ADD COLUMN IF NOT EXISTS normal_braking INT64;"""
    run_sql(sql)
    for table_name in table_names:
        sql = f"""UPDATE `{project_id}.{tables}.{table}`
                            SET normal_braking = (
                            select count(linear_g) from `{project_id}.{data}.{table_name}` WHERE (linear_g < 0.1 AND linear_g > -0.1)
                            )
                            WHERE vehicleid = {table_name};
                        """
        results = run_sql(sql)

def insert_harsh_braking():
    table_names = select_table_names()
    sql = f"""Alter table {project_id}.{tables}.{table} ADD COLUMN IF NOT EXISTS harsh_braking INT64;"""
    run_sql(sql)
    for table_name in table_names:
        sql = f"""UPDATE `{project_id}.{tables}.{table}`
                            SET harsh_braking = (
                            select count(linear_g) from `{project_id}.{data}.{table_name}` WHERE ((linear_g < 0.3 AND linear_g > -0.3) AND NOT (linear_g < 0.1 AND linear_g > -0.1))
                            )
                            WHERE vehicleid = {table_name}
                        """
        results = run_sql(sql)

def insert_emergency_braking():
    table_names = select_table_names()
    sql = f"""Alter table {project_id}.{tables}.{table} ADD COLUMN IF NOT EXISTS emergency_braking INT64;"""
    run_sql(sql)
    for table_name in table_names:
        sql = f"""UPDATE `{project_id}.{tables}.{table}`
                            SET emergency_braking = (
                            select count(linear_g) from `{project_id}.{data}.{table_name}` WHERE ((linear_g < 0.5 AND linear_g > -0.5) AND NOT (linear_g < 0.3 AND linear_g > -0.3))
                            )
                            WHERE vehicleid = {table_name};
                        """
        results = run_sql(sql)

def insert_critical_braking():
    table_names = select_table_names()
    sql = f"""Alter table {project_id}.{tables}.{table} ADD COLUMN IF NOT EXISTS CriticalBraking INT64;"""
    run_sql(sql)
    for table_name in table_names:
        sql = f"""UPDATE `{project_id}.{tables}.{table}`
                            SET CriticalBraking = (
                            select count(linear_g) from `{project_id}.{data}.{table_name}` WHERE (linear_g > 0.5) OR (linear_g < -0.5)
                            )
                            WHERE vehicleid = {table_name};
                        """
        results = run_sql(sql)

def insert_idle_time():
    table_names = select_table_names()
    sql = f"""ALTER TABLE {project_id}.{tables}.{table} ADD COLUMN IF NOT EXISTS idle_ratio FLOAT64;"""
    run_sql(sql)
    for table_name in table_names:
        sql = f"""
                UPDATE `{project_id}.{tables}.{table}`
                SET idle_ratio = (
                    SELECT idle / total as idle_ratio
                    FROM (
                        SELECT
                            SUM(CASE WHEN event_description LIKE 'Idle%' THEN 1 ELSE 0 END) AS idle,
                            COUNT(*) AS total
                        FROM `{project_id}.{data}.{table_name}`
                    ) subquery
                )
                WHERE vehicleid = {table_name};
                """
        results = run_sql(sql)

def insert_average_distance_stops():
    sql = f"""alter table `hackathon_tables.{table}` add column if not exists bad_driver BOOL;
                alter table `hackathon_tables.{table}` add column if not exists average_distance_per_stop FLOAT64;
                alter table `hackathon_tables.{table}` add column if not exists average_number_of_stops_per_day FLOAT64;
            update `hackathon_tables.{table}` as driver
            set driver.bad_driver = updated.bad_driver,
            driver.average_distance_per_stop = updated.average_distance_per_stop,
            driver. average_number_of_stops_per_day = updated.average_number_of_stops_per_day
            from `hackathon-1123-395609.hackathon_tables.updated_stops` as updated
            where driver.vehicleid=updated.vehicleid"""
    run_sql(sql)

def insert_classified_time_of_day():
    run_sql(f"""Alter table `{project_id}.{tables}.{table}` ADD COLUMN IF NOT EXISTS dangerousTimes INT64""")
    table_names = select_table_names()
    for table_name in table_names:
        results = run_sql(f"""UPDATE `{project_id}.{tables}.{table}`
                            SET dangerousTimes = (
                            SELECT COUNT(*)
                            FROM `{project_id}.{data}.{table_name}`
                            WHERE EXTRACT(HOUR FROM timestamp) >= 0
                            AND EXTRACT(HOUR FROM timestamp) < 4
                            )
                            WHERE vehicleid = {table_name};
                        """)
    # Print the query results
        for row in results:
            print("Row:", row)

def insert_classified_area():
    run_sql(f"""Alter table `{project_id}.{tables}.{table}` ADD COLUMN IF NOT EXISTS DangerousAreas INT64""")
    table_names = select_table_names()
    for table_name in table_names:
        results = run_sql(f"""UPDATE `{project_id}.{tables}.{table}`
                            SET DangerousAreas = (
                            SELECT COUNT(*)
                            FROM `{project_id}.{data}.{table_name}`
                            WHERE MP_NAME IN ('Nyanga', 'Khayelitsha', 'Delft', 'Mitchells Plain', 'Langa', 'Gugulethu', 'Manenberg', 'Bishop Lavis', 'Hanover park')
                            )
                            WHERE vehicleid = {table_name};
                        """)

def insert_average_nmr_stops_per_day():
    run_sql(f"""Alter table `{project_id}.{tables}.{table}` ADD COLUMN IF NOT EXISTS NmrOfStops INT64""")
    table_names = select_table_names()
    for table_name in table_names:
        results = run_sql(f"""UPDATE `{project_id}.{tables}.{table}` AS d
                            SET NmrOfStops = (
                                SELECT COUNT(*)
                                FROM (
                                    SELECT
                                        timestamp,
                                        ignitionState,
                                        LAG(ignitionState, 1, '') OVER (PARTITION BY vehicleid ORDER BY timestamp) AS prev_ignitionState
                                    FROM
                                        `{project_id}.{data}.{table_name}`
                                ) subquery
                                WHERE d.vehicleid = subquery.vehicleid AND d.timestamp = subquery.timestamp AND d.ignitionState = 'ON' AND subquery.prev_ignitionState <> 'ON'
                            );
                            """)
    for row in results:
            print("Row:", row)

def insert_speed_penalty():
    table_names = select_table_names()
    run_sql(f"""Alter table {project_id}.{tables}.{table} ADD COLUMN IF NOT EXISTS SpeedPenalty FLOAT64;""")
    for table_name in table_names:
        results = run_sql(f"""UPDATE `{project_id}.{tables}.{table}`
                            SET SpeedPenalty =
                            (
                              select COALESCE(sum(penalty_speed),0) from
                              (
                                SELECT vehicleid, speed, road_speed, (speed/road_speed-1)*100 as penalty_speed
                                FROM `{project_id}.{data}.{table_name}`
                                WHERE speed > road_speed
                              )
                            )
                            WHERE vehicleid = {table_name};
                    """)

def insert_braking_penalty():
    table_names = select_table_names()
    run_sql(f"""Alter table {project_id}.{tables}.{table} ADD COLUMN IF NOT EXISTS BrakingPenalty FLOAT64;""")
    for table_name in table_names:
        results = run_sql(f"""UPDATE `{project_id}.{tables}.{table}`
                            SET BrakingPenalty =
                            (
                              select COALESCE(sum(brake_ratio),0) from
                                (
                                  SELECT ((-linear_g/0.4)-1)*100 as brake_ratio
                                  FROM `{project_id}.{data}.{table_name}`
                                  WHERE linear_g < -0.4
                                )
                            )
                            WHERE vehicleid = {table_name};
                    """)

def insert_acceleration_penalty():
    table_names = select_table_names()
    run_sql(f"""Alter table {project_id}.{tables}.{table} ADD COLUMN IF NOT EXISTS AccelerationPenalty FLOAT64;""")
    for table_name in table_names:
        results = run_sql(f"""UPDATE `{project_id}.{tables}.{table}`
                            SET AccelerationPenalty =
                            (
                              select COALESCE(sum(acc_ratio),0) from
                              (
                                SELECT ((linear_g/0.4)-1)*100 as acc_ratio
                                FROM `{project_id}.{data}.{table_name}`
                                WHERE linear_g > 0.4
                              )
                            )
                            WHERE vehicleid = {table_name};
                    """)

def insert_net_acceleration_penalty():
    table_names = select_table_names()
    run_sql(f"""Alter table {project_id}.{tables}.{table} ADD COLUMN IF NOT EXISTS NetAccelerationPenalty FLOAT64;""")
    for table_name in table_names:
        results = run_sql(f"""UPDATE `{project_id}.{tables}.{table}`
                            SET NetAccelerationPenalty =
                            (
                              select COALESCE(sum(net_g),0) from
                              (
                                select sqrt(power(x_accel, 2)+power(y_accel, 2)+power(z_accel, 2))/9.81 as net_g
                                FROM `{project_id}.{data}.{table_name}`
                              ) where net_g > 0.4
                            )
                            WHERE vehicleid = {table_name};
                    """)

# todo: do same as normal speed penalty
def insert_corner_penalty():
    table_names = select_table_names()
    for table_name in table_names:
        results = run_sql(f"""Alter table {project_id}.{tables}.{table} ADD COLUMN IF NOT EXISTS CornerSpeedingPenalty FLOAT64;
                            UPDATE `{project_id}.{tables}.{table}`
                            SET CornerSpeedingPenalty = (
                                SELECT COALESCE(SUM((speed/road_speed-1)*100), 0.0)
                                FROM `{project_id}.{data}.{table_name}`
                                WHERE speed > road_speed
                                AND event_description = 'Corner'
                            )
                            WHERE vehicleid = {table_name};
                            """)

In [1]:
if __name__ == "__main__":
  # print("create driver")
  # create_driver()
  # print("vehicle id")
  # insert_vehicleid()
  # print("over speed")
  # insert_over_speed_limit()
  # print("gps lost")
  # insert_gps_lost_count()
  # print("telematics off")
  # insert_telematics_off_count()
  # print("normal")
  # insert_normal_braking()
  # print("harsh")
  # insert_harsh_braking()
  # print("emergency")
  # insert_emergency_braking()
  # print("critical")
  # insert_critical_braking()
  # print("idle")
  # insert_idle_time()
  # print("time of day")
  # insert_classified_time_of_day()
  # print("area")
  # insert_classified_area()
  # print("speed penalty")
  # insert_speed_penalty()
  # print("braking penalty")
  # insert_braking_penalty()
  # print("acc penalty")
  # insert_acceleration_penalty()
  # print("net acc penalty")
  # insert_net_acceleration_penalty()
  # print("corner penalty")
  # insert_corner_penalty()
  # print("claims")
  # insert_average_distance_stops()

SyntaxError: ignored